# FM

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
from sklearn.datasets import make_circles

# 加载同心圆数据
xs, ys = make_circles(600, noise=0.1, factor=0.2)
ys = ys * 2 - 1  # 0/1转换为-1/1

# 特征维数
FEAT_DIM = 20
# 构造噪声特征
xs = np.concatenate([xs, np.random.normal(0.0, 0.01, (600, FEAT_DIM - 2))], axis=1)
# 隐藏向量维度
HIDDEN_DIM = 2

print(xs.shape, ys.shape)
print(xs[0], ys[0])

(600, 20) (600,)
[ 0.79804525  0.61588447 -0.01379158 -0.00101946  0.0078949   0.0029288
 -0.01874751 -0.00496474 -0.0175083   0.00480337  0.02237244 -0.00208155
  0.00837737 -0.02109492 -0.00208576 -0.00523745  0.00973908 -0.00609149
  0.01135981  0.01070201] -1


In [3]:
# 训练
from mflow import core, ops, opts

# 超参数
lr = 0.001
epoch = 50
batch_size = 16

with core.NameScope("FM"):
    # 初始化变量
    x = core.Variable(size=(FEAT_DIM, 1), trainable=False)
    y = core.Variable(size=(1, 1), trainable=False)
    w = core.Variable(size=(1, FEAT_DIM), trainable=True)
    h = core.Variable(size=(HIDDEN_DIM, FEAT_DIM), trainable=True)
    b = core.Variable(size=(1, 1), trainable=True)
    # 模型定义
    pred = ops.Add(
        ops.MatMul(
            ops.Reshape(x, shape=(1, FEAT_DIM)), 
            ops.MatMul(ops.MatMul(ops.Reshape(h, shape=(FEAT_DIM, HIDDEN_DIM)), h), x)
        ),  # 二次项
        ops.MatMul(w, x),  # 一次项
        b  # 偏置项
    )
    predicter = ops.Logistic(pred)
    loss = ops.loss.LogLoss(ops.Multiply(y, pred))
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).T)
            y.setValue(np.mat(lab))
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).T)
            predicter.forward()
            preds.append(predicter.value[0, 0])  # 结果
        preds = (np.array(preds) > 0.5).astype("int") * 2 - 1
        acc = (ys == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.500.
Epoch: 2, acc: 0.500.
Epoch: 3, acc: 0.500.
Epoch: 4, acc: 0.505.
Epoch: 5, acc: 0.587.
Epoch: 6, acc: 0.747.
Epoch: 7, acc: 0.823.
Epoch: 8, acc: 0.868.
Epoch: 9, acc: 0.903.
Epoch: 10, acc: 0.928.
Epoch: 11, acc: 0.948.
Epoch: 12, acc: 0.955.
Epoch: 13, acc: 0.957.
Epoch: 14, acc: 0.958.
Epoch: 15, acc: 0.962.
Epoch: 16, acc: 0.967.
Epoch: 17, acc: 0.972.
Epoch: 18, acc: 0.973.
Epoch: 19, acc: 0.973.
Epoch: 20, acc: 0.977.
Epoch: 21, acc: 0.978.
Epoch: 22, acc: 0.980.
Epoch: 23, acc: 0.980.
Epoch: 24, acc: 0.982.
Epoch: 25, acc: 0.982.
Epoch: 26, acc: 0.983.
Epoch: 27, acc: 0.983.
Epoch: 28, acc: 0.987.
Epoch: 29, acc: 0.988.
Epoch: 30, acc: 0.988.
Epoch: 31, acc: 0.988.
Epoch: 32, acc: 0.988.
Epoch: 33, acc: 0.990.
Epoch: 34, acc: 0.992.
Epoch: 35, acc: 0.992.
Epoch: 36, acc: 0.992.
Epoch: 37, acc: 0.992.
Epoch: 38, acc: 0.992.
Epoch: 39, acc: 0.992.
Epoch: 40, acc: 0.995.
Epoch: 41, acc: 0.997.
Epoch: 42, acc: 0.997.
Epoch: 43, acc: 0.997.
Epoch: 44, acc: 0.99